# Indicadores DIT

## 1. Importación de librerías y bases de datos

In [1]:
#Librerías
import pandas as pd
import os
import numpy as np

In [5]:
os.chdir(r'C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\03. ENDES\1. Data')

# Modulo DIT
dit = pd.DataFrame()
for anio in range(2019,2023):
    df = pd.read_stata(f"dit_{anio}.dta", convert_categoricals=True)
    df.columns = df.columns.str.lower()
    dit= pd.concat([dit,df], ignore_index=True)
dit['id1'] = dit['id1'].astype(int)

# Modulo REC21
rec21 = pd.DataFrame()
for anio in range(2019,2023):
    try:
        df = pd.read_stata(f"rec21_{anio}.dta", columns = ['ID1', 'CASEID', 'BIDX', 'B3', 'B4'])
    except:
        df = pd.read_stata(f"rec21_{anio}.dta", columns = ['id1', 'caseid', 'bidx', 'b3', 'b4'])
    df.columns = df.columns.str.lower()
    rec21 = pd.concat([rec21,df], ignore_index=True)

# Modulo REC0111
rec0111 = pd.DataFrame()
for anio in range(2019,2023):
    try:
        df = pd.read_stata(f"rec0111_{anio}.dta", convert_categoricals=True, columns = ['ID1', 'CASEID', 'V001', 'V005', 'V008', 'V012', 'V022' ,'V024' ,'V025' ,'V149' ,'V190'])
    except:
        df = pd.read_stata(f"rec0111_{anio}.dta", convert_categoricals=True, columns = ['id1','caseid', 'v001', 'v005', 'v008', 'v012', 'v022' ,'v024' ,'v025' ,'v149' ,'v190'])
    df.columns = df.columns.str.lower()
    rec0111= pd.concat([rec0111,df], ignore_index=True)

# Modulo REC91
rec91 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'CASEID', 'SREGION', 'S119', 'S108N'])
    except:
        df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['id1','caseid','sregion', 's119', 's108n'])
    df.columns = df.columns.str.lower()

    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rec91 = pd.concat([rec91, df], ignore_index=True)

# Modulo REC42
rec42 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rec42_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'CASEID', 'V463A'])
    except:
        df = pd.read_stata(f"rec42_{anio}.dta", convert_categoricals=True, columns= ['id1','caseid','v463a'])
    df.columns = df.columns.str.lower()

    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rec42 = pd.concat([rec42, df], ignore_index=True)

# Modulo REC41
rec41 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rec41_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'CASEID', 'MIDX','M19','M18'])
        df
    except:
        df = pd.read_stata(f"rec41_{anio}.dta", convert_categoricals=True, columns= ['id1','caseid', 'midx','m19','m18'])
    df.columns = df.columns.str.lower()
    df['bidx'] = df['midx']
    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rec41 = pd.concat([rec41, df], ignore_index=True)


# Modulo rech23
rech23 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rech23_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'HHID', 'HV270'])
    except:
        df = pd.read_stata(f"rech23_{anio}.dta", convert_categoricals=True, columns= ['id1','hhid', 'hv270'])
    df.columns = df.columns.str.lower()
    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rech23 = pd.concat([rech23, df], ignore_index=True)

# Modulo rech0
rech0 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rech0_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'HHID', 'HV001', 'HV025', 'HV040', 'HV022'])
    except:
        df = pd.read_stata(f"rech0_{anio}.dta", convert_categoricals=True, columns= ['id1','hhid', 'hv001', 'hv025', 'hv040', 'hv022'])
    df.columns = df.columns.str.lower()

    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rech0 = pd.concat([rech0, df], ignore_index=True)

# Modulo rech1
rech1 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rech1_{anio}.dta", convert_categoricals=True, columns= ['ID1','HHID', 'HVIDX', 'HV103'])
    except:
        df = pd.read_stata(f"rech1_{anio}.dta", convert_categoricals=True, columns= ['id1','hhid', 'hvidx','hv103'])
    df.columns = df.columns.str.lower()
    df['hc0'] = df['hvidx']
    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rech1 = pd.concat([rech1, df], ignore_index=True)
    

# Modulo rech6
rech6 = pd.DataFrame()
for anio in range(2019, 2023):
    try:
        df = pd.read_stata(f"rech6_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'HHID', 'HC0','HC1', 'HC53', 'HC56', 'HC57', 'HC70'])
    except:
        df = pd.read_stata(f"rech6_{anio}.dta", convert_categoricals=True, columns= ['id1','hhid', 'hc0','hc1', 'hc53', 'hc56', 'hc57', 'hc70'])
    df.columns = df.columns.str.lower()

    # Exclude empty or all-NA columns before concatenation
    df = df.dropna(axis=1, how='all')
    rech6 = pd.concat([rech6, df], ignore_index=True)
    

C:\Users\User\AppData\Local\Temp\ipykernel_5788\4166083060.py:35: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['ID1', 'CASEID', 'SREGION', 'S119', 'S108N'])
C:\Users\User\AppData\Local\Temp\ipykernel_5788\4166083060.py:37: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(f"rec91_{anio}.dta", convert_categoricals=True, columns= ['id1','caseid','sregion', 's119', 's108n'])
C:\Users\User\AppData\Local\Temp\ipyke

In [6]:
# Merging datasets
dfDIT = pd.merge(dit, rec21, how='inner', on=['id1','caseid', 'bidx'], validate="1:1")
dfDIT = pd.merge(dfDIT, rec0111, how='inner', on=['id1','caseid'], validate="m:1")
dfDIT = pd.merge(dfDIT, rec91, how='inner', on=['id1','caseid'], validate="m:1")
dfDIT = pd.merge(dfDIT, rec41, how='inner', on=['id1','caseid', 'bidx'], validate="1:1")
dfDIT = pd.merge(dfDIT, rec42, how='inner', on=['id1','caseid'], validate="m:1")

dfBajoPeso = pd.merge(rec0111, rec91, how='inner', on=['id1','caseid'], validate="1:1")
dfBajoPeso = pd.merge(dfBajoPeso, rec42, how='inner', on=['id1','caseid'], validate="1:1")
df = pd.merge(rec21, rec41, how='inner', on=['id1','caseid', 'bidx'], validate="1:1")
dfBajoPeso = pd.merge(dfBajoPeso, df, how='inner', on=['id1','caseid'], validate="1:m")

df1 = pd.merge(rech23, rech0, how='inner', on=['id1','hhid'], validate="1:1")
df2 = pd.merge(rech1, rech6, how='inner', on=['id1','hhid', 'hc0'], validate="1:1")
dfAnemiaDCI = pd.merge(df2, df1, how='inner', on=['id1','hhid'], validate="m:1")


del dit, df, rec0111, rec21, rec91, anio, rec42, rec41, df1, df2, rech0, rech1, rech23, rech6

## 2. Generación de variables

### 2.1. Características del hogar o entorno

In [8]:
# Generando variables de características del hogar/madre

# Educación de la madre
dfDIT['v149_'] = dfDIT['v149'].array.codes
dfDIT['eduComp'] = 0
dfDIT.loc[dfDIT['v149_'] >= 2, 'eduComp'] = 1
dfDIT.loc[dfDIT['v149_'] >= 4, 'eduComp'] = 2
dfDIT.loc[dfDIT['v149_'] == 5, 'eduComp'] = 3
del dfDIT['v149_']
# Labels for eduComp
labels = {1: "Sin nivel educativo", 2: "Primaria completa", 3: "Secundaria completa", 4: "Superior"}
dfDIT["eduComp"] = dfDIT["eduComp"].map(labels)

# Generamos la variable edadm (edad en meses)
dfDIT["edadm"] = dfDIT["v008"] - dfDIT["b3"]
dfDIT["e_0a35"] = dfDIT["edadm"] < 36
dfDIT["e_0a59"] = dfDIT["edadm"] < 60

### 2.2. R1: Nacimiento saludable

In [50]:
# Peso al nacer
dfDIT["pesonac"] = 1
dfDIT.loc[(dfDIT["m19"] == 'No se pesó al nacer'), 'm19'] = 9996
dfDIT.loc[(dfDIT["m19"] == 'No sabe'), 'm19'] = 9998
dfDIT.loc[(dfDIT["m19"] > 2499) & (dfDIT["m19"] <= 8000) & (dfDIT["v012"] > 14), "pesonac"] = 2
dfDIT.loc[dfDIT["m19"] == 9996 & (dfDIT["v012"] > 14), "pesonac"] = 3
dfDIT.loc[dfDIT["m19"] == 9998 & (dfDIT["v012"] > 14), "pesonac"] = 4

# Labels for pesonac
labels = {1: "Menos de 2.5kg", 2: ">=2.5 kg", 3: "No pesados al nacer", 4: "No sabe, sin informacion"}
dfDIT["pesonac"] = dfDIT["pesonac"].map(labels)

# Nacimiento prematuro --> 
# Q220A Duración del embarazo
# QULT5 Es un nacimiento en ultimos 5 años

# Prevalencia de anemia en mujeres gestantes
# - Denominador. Constituido por el total gestantes que al momento de la aplicación de la encuesta cumplen con la definición de pernoctaron en el hogar la noche anterior a la encuesta.
# - Numerador. Constituido por gestantes que cumplen con la definición del denominador y presentan anemia.

In [60]:
dfDIT["pesonac"].value_counts()

pesonac
>=2.5 kg          59021
Menos de 2.5kg     6089
Name: count, dtype: int64

### 2.3. R2: Apego seguro 

In [151]:
## Porcentaje de madres de niñas y niños entre 9 y 12 meses de edad que responde de manera inmediata y adecuada al llanto de su hija/o
dfDIT['sensMadre9a12'] = 0
dfDIT['qi478'] = dfDIT['qi478'].values
cond = (dfDIT['qi478'] >= 9) & (dfDIT['qi478'] <= 12)
cond1 = (dfDIT['qi478e10'] == "Deja de hacer lo que está haciendo y la/lo atiende") | (dfDIT['qi478e10'] ==  "Le pide otra persona que la(o) atienda") | (dfDIT['qi478e10'] == "La (Lo) atiende aun cuando siga haciendo sus labores")
dfDIT.loc[ cond & cond1 ,'sensMadre9a12'] = 1
dfDIT.loc[-cond,'sensMadre9a12'] = np.nan

## Porcentaje de niños y niñas entre 9 y 12 meses de edad que tiene adecuada interacción madre-hijo/a como precursor del apego seguro
dfDIT['apego9a12'] = 0
dfDIT['apego9a12a'] = 0
dfDIT['apego9a12b'] = 0
dfDIT['apego9a12c'] = 0
cond1 = (dfDIT['qi478e7'] == "No") 
cond2 = (dfDIT['qi478e8'] == "No") 
cond3 = (dfDIT['qi478e9'] == "No")

# Apego seguro - dummy
dfDIT.loc[ cond & cond1 & cond2 & cond3 ,'apego9a12'] = 1
dfDIT.loc[-cond,'apego9a12'] = np.nan

# Apego seguro - categórica
dfDIT.loc[ cond & cond1 ,'apego9a12a'] = 1
dfDIT.loc[-cond,'apego9a12a'] = np.nan
dfDIT.loc[ cond & cond2 ,'apego9a12b'] = 1
dfDIT.loc[-cond,'apego9a12b'] = np.nan
dfDIT.loc[ cond & cond3 ,'apego9a12c'] = 1
dfDIT.loc[-cond,'apego9a12c'] = np.nan
dfDIT['apego9a12_cat'] = dfDIT['apego9a12a'] + dfDIT['apego9a12b'] + dfDIT['apego9a12c']
dfDIT.loc[-cond,'apego9a12_cat'] = np.nan

del dfDIT['apego9a12a'] , dfDIT['apego9a12b'] , dfDIT['apego9a12c']

In [ ]:
# Porcentaje de niñas y niños entre 24 y 71 meses de edad cuya madre no ejerce conductas de castigo físico hacia su hija/o
# - Numerador: Número niñas y niños niños de 24 a 71 meses cuyas madres participaron de la encuesta y declararon que no ejercen conductas de castigo físico hacia su hija/o.
# - Denominador: Número niñas y niños niños de 24 a 71 meses cuyas madres participaron de la encuesta y respondieron la pregunta sobre si ejercen conductas de castigo físico hacia su hija/o.

In [134]:
dfDIT['apego9a12_cat'].value_counts()

apego9a12_cat
3.0    2331
2.0    1841
1.0     678
0.0     399
Name: count, dtype: int64

### 2.4. R3: Adecuado estado nutricional 


In [41]:
# Prevalencia de desnutrición crónica en niños y niñas menores de 60 meses
dfAnemiaDCI['desnwho'] = pd.Series(dtype=float)
dfAnemiaDCI.loc[(dfAnemiaDCI['hc70'] == 'Altura fuera de los límites plausibles.'), 'hc70'] = 9996
dfAnemiaDCI.loc[(dfAnemiaDCI['hc70'] == 'Edad en días fuera de límites plausibles'), 'hc70'] = 9997
dfAnemiaDCI.loc[(dfAnemiaDCI['hc70'] == 'Casos marcados'), 'hc70'] = 9998
dfAnemiaDCI['hc70'] = dfAnemiaDCI['hc70'].values
dfAnemiaDCI.loc[(dfAnemiaDCI['hc70'] < -200) & (dfAnemiaDCI['hv103'] =='Sí'), 'desnwho'] = 1
dfAnemiaDCI.loc[(dfAnemiaDCI['hc70'] >= -200) & (dfAnemiaDCI['hc70'] < 601) & (dfAnemiaDCI['hv103'] =='Sí'), 'desnwho'] = 0
dfAnemiaDCI.loc[(dfAnemiaDCI['hc70'] == 9996) | (dfAnemiaDCI['hc70'] == 9997) | (dfAnemiaDCI['hc70'] == 9998), 'desnwho'] = np.nan

# Labels for desnwho "desnutricion cronica total OMS"
labels = {0: "Sin desnutricion cronica", 1: "Con desnutricion cronica"}
dfAnemiaDCI["desnwho"] = dfAnemiaDCI["desnwho"].map(labels)

# Prevalencia de desnutrición crónica severa en niños y niñas menores de 60 meses
dfAnemiaDCI['desn_sev'] = pd.Series()
dfAnemiaDCI.loc[(dfAnemiaDCI['hc70'] < -300) & (dfAnemiaDCI['hv103'] == 'Sí'), 'desn_sev'] = 1
dfAnemiaDCI.loc[(dfAnemiaDCI['hc70'] >= -300 & dfAnemiaDCI['hc70'] < 601) & (dfAnemiaDCI['hv103'] == 'Sí'), 'desn_sev'] = 0

# Labels for desn_sev "desnutricion cronica severa OMS"
labels = {0: "Sin desnutricion cronica severa", 1: "Con desnutricion cronica severa"}
dfAnemiaDCI["desn_sev"] = dfAnemiaDCI["desn_sev"].map(labels)




TypeError: Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]

In [37]:
dfAnemiaDCI['desnwho'].value_counts()

desnwho
0.0    67302
1.0     9861
Name: count, dtype: int64

In [ ]:
# Prevalencia de anemia en niños y niñas de 6 a 23 meses
dfAnemiaDCI['alt'] = dfAnemiaDCI['hv040']/1000 * 3.3
#Se ajusta el nivel de hemoglobina segun la altitud (alt)
dfAnemiaDCI['haj'] = dfAnemiaDCI['hc53']/10 -(-0.032*dfAnemiaDCI['alt'] + 0.022*dfAnemiaDCI['alt']*dfAnemiaDCI['alt'])
dfAnemiaDCI['anemia'] = pd.Series()
dfAnemiaDCI.loc[(dfAnemiaDCI['haj']>1   & dfAnemiaDCI['haj']<11) &  dfAnemiaDCI['hv103']==1 & (dfAnemiaDCI['hc1']>5 & dfAnemiaDCI['hc1']<36), 'anemia'] = 1
dfAnemiaDCI.loc[(dfAnemiaDCI['haj']>=11   & dfAnemiaDCI['haj']<94) &  dfAnemiaDCI['hv103']==1 & (dfAnemiaDCI['hc1']>5 & dfAnemiaDCI['hc1']<36), 'anemia'] = 1

# Labels for anemia 
labels = {0: "No tiene anemia", 1: "Tiene anemia"}
dfDIT["anemia"] = dfDIT["anemia"].map(labels)


# Niveles de anemia
dfAnemiaDCI['nivAnemia'] = pd.Series()
dfAnemiaDCI.loc[(dfAnemiaDCI['hv103']==1) & (dfAnemiaDCI['hc1']>5 & dfAnemiaDCI['hc1']<36),'nivAnemia'] = dfAnemiaDCI['hc57']


In [ ]:

# Prevalencia de sobrepeso y obesidad en menores 36 a 59 meses de edad
# - Denominador o universo. Constituido por todos los niños y niñas entre 36 y 60 meses de edad que al momento de la aplicación de la encuesta cumplen con la definición de residente habitual del hogar. Se incluye a los niños y niñas: i) presentes en el hogar independiente de la presencia o ausencia de la madre; ii) con valores validos de la talla y peso al momento de aplicar la encuesta; iii) con valores del índice antropométrico en el rango de aceptable (-6<z<6).
# - Numerador. Constituido por todos los niños y niñas entre 36 y 60 meses que, además de cumplir con los criterios definidos en el denominador, presentan sobrepeso.

### 2.5. R4: Comunicación verbal efectiva

In [150]:
#Porcentaje de niñas y niños de 9 a 36 meses con comunicación verbal a nivel comprensivo y expresivo apropiada para su edad
for grupo in ['9a12','13a18', '19a23', '24a36']:
    dfDIT[f'comEfec{grupo}'] = 0
    dfDIT[f'comEfec{grupo}a'] = 0
    dfDIT[f'comEfec{grupo}b'] = 0
    dfDIT[f'comEfec{grupo}c'] = 0


dfDIT['qi478'] = dfDIT['qi478'].values
cond9a12 = (dfDIT['qi478'] >= 9) & (dfDIT['qi478'] <= 12) 
cond9a12a = (dfDIT['qi478e3'] == "Si")
cond9a12b = (dfDIT['qi478e4'] == "Si")
cond9a12c = (dfDIT['qi478e5'] == "Si")
cond13a18 = (dfDIT['qi478'] >= 13) & (dfDIT['qi478'] <= 18)
cond13a18a = (dfDIT['qi478f3'] == "Si")
cond13a18b = (dfDIT['qi478f4'] == "Si")
cond13a18c = (dfDIT['qi478f5'] == "Si")
cond19a23 = (dfDIT['qi478'] >= 19) & (dfDIT['qi478'] <= 23)
cond19a23a = (dfDIT['qi478g1'] == "Si")
cond19a23b = ((dfDIT['qi478g2_a'] == "Si") & (dfDIT['qi478g2_b'] == "Si") & (dfDIT['qi478g2_c'] == "Si"))
cond19a23c = (dfDIT['qi478g3'] == "Si")
cond24a36 = (dfDIT['qi478'] >= 24) & (dfDIT['qi478'] <= 36)
cond24a36a = (dfDIT['qi478h1'] == "Si")
cond24a36b = (dfDIT['qi478h2'] == "Si") 
cond24a36c = (dfDIT['qi478h3'] == "Si")

for grupo in ['9a12','13a18', '19a23', '24a36']:
    # Comunicacion efectiva - dummy por edad
    dfDIT.loc[ locals()[f'cond{grupo}'] & locals()[f'cond{grupo}a'] & locals()[f'cond{grupo}b'] & locals()[f'cond{grupo}c'] ,f'comEfec{grupo}'] = 1
    dfDIT.loc[-locals()[f'cond{grupo}'],f'comEfec{grupo}'] = np.nan

    # Comunicacion efectiva - categórica por edad
    dfDIT.loc[ locals()[f'cond{grupo}'] & locals()[f'cond{grupo}a'] ,f'comEfec{grupo}a'] = 1
    dfDIT.loc[-locals()[f'cond{grupo}'],f'comEfec{grupo}a'] = np.nan
    dfDIT.loc[ locals()[f'cond{grupo}'] & locals()[f'cond{grupo}b'] ,f'comEfec{grupo}b'] = 1
    dfDIT.loc[-locals()[f'cond{grupo}'],f'comEfec{grupo}b'] = np.nan
    dfDIT.loc[ locals()[f'cond{grupo}'] & locals()[f'cond{grupo}c'] ,f'comEfec{grupo}c'] = 1
    dfDIT.loc[-locals()[f'cond{grupo}'],f'comEfec{grupo}c'] = np.nan
    dfDIT[f'comEfec{grupo}_cat'] = dfDIT[f'comEfec{grupo}a'] + dfDIT[f'comEfec{grupo}b'] + dfDIT[f'comEfec{grupo}c']
    dfDIT.loc[-locals()[f'cond{grupo}'], f'comEfec{grupo}_cat'] = np.nan
    
    del dfDIT[f'comEfec{grupo}a'] , dfDIT[f'comEfec{grupo}b'] , dfDIT[f'comEfec{grupo}c']


for var_name in list(locals()):
    if var_name.startswith('cond'):
        del locals()[var_name]

In [ ]:
# Porcentaje de madres de niñas y niños entre 9 y 18 meses de edad que verbaliza las acciones que realiza con su hija/o mientras las desarrolla
# - Numerador: Número de madres de niñas y niños de 9 a 18 meses que participaron de la encuesta y declararon que sí verbalizan las acciones que realiza con su hija/o mientras las desarrolla.
# - Denominador: Número de madres de niñas y niños de 9 a 18 meses que participaron de la encuesta y respondieron la pregunta sobre si verbalizan las acciones que realiza con su hija/o mientras las desarrolla.

In [ ]:
# Porcentaje de niñas y niños entre 19 y 36 meses de edad que participa de forma frecuente en conversaciones con adultos
# - Numerador: Número de niñas y niños de 19 a 36 meses cuyas madres participaron de la encuesta y declararon que su hija/o sí participa de forma frecuente en las conversaciones con adultos.
# - Denominador: Número de niñas y niños de 19 a 36 meses cuyas madres participaron de la encuesta y respondieron la pregunta sobre si su hija/o participa de forma frecuente en las conversaciones con adultos.

### 2.6. R5: Camina solo

In [ ]:
# Porcentaje de niñas y niñas entre 15 a 18 meses que caminan por propia iniciativa sin necesidad de detenerse para lograr el equilibrio
# A: Número de niñas y niños de 15 a 18 meses cuya madre reporta que la/el niña/o camina solo con soltura.
# B: Número de niñas y niños de 15 a 18 meses con información completa en el tramo 2.

# Porcentaje de niñas y niños de 9 a 12 meses de edad que tiene, en el hogar, un espacio físico para desplazarse/caminar libremente
# - Numerador: Número de niñas y niños niños de 9 a 12 meses cuyas madres participaron de la encuesta y declararon que su hija/o sí tiene un espacio con piso firme y sin objetos en el que pueda desplazarse/caminar libremente.
# - Denominador: Número de niñas y niños niños de 9 a 12 meses cuyas madres participaron de la encuesta y respondieron la pregunta sobre si su hija/o tiene un espacio sin objetos en el que pueda desplazarse/caminar libremente.

# Porcentaje de niñas y niños de 13 a 18 meses de edad que tiene, en el hogar, un espacio de juego libre de peligros para su integridad física
# - Numerador: Número de niñas y niños niños de 13 a 18 meses cuyas madres participaron de la encuesta y declararon que su hija/o sí tiene un espacio de juego libre de peligros.
# - Denominador: Número de niñas y niños niños de 13 a 18 meses cuyas madres participaron de la encuesta y respondieron la pregunta sobre si su hija/o tiene de juego libre de peligros.

### 2.7. R6: Regulación y control de emociones

In [ ]:
# Porcentaje de niñas y niños de 24 a 71 meses que regula sus emociones y comportamientos en situación de frustración y establecimiento de límites
# A: Número de niñas y niños de 24 a 71 meses cuya madre reporta que no lloran, gritan o hacen berrinche la mayor parte del tiempo
# B: Número de niñas y niños de 24 a 71 meses cuya madre reporta que esperan tranquilos cuando quieren algo y ellas les dicen que esperen
# C: Número de niñas y niños de 24 a 71 meses cuya madre reporta que no se hacen daño o agreden a sí mismos o a los demás cuándo quieren algo y ellas les dicen que NO



### 2.8. R7: Función simbólica

In [ ]:
# Porcentaje de niñas y niños de 55 a 71 meses de edad que representan sus vivencias a través del juego y el dibujo
# A: Número de niñas y niños de 55 a 71 meses cuyas madres reportan que pueden dibujar una figura humana con detalles
# B: Número de niñas y niños de 55 a 71 meses cuya madre reporta que representan sus vivencias a través del juego


# Porcentaje de niñas y niños entre 24 y 71 meses de edad que tienen en casa materiales de juego estructurados y no estructurados
# - Numerador: Número de niñas y niños niños de 24 a 71 meses cuyas madres participaron de la encuesta y declararon que su hija/o sí tienen en casa materiales de juego estructuraos y no estructurados.
# - Denominador: Número de niñas y niños niños de 24 a 71 meses cuyas madres participaron de la encuesta y respondieron la pregunta sobre si su hija/o tienen en casa materiales de juego estructuraos y no estructurados.

## 3. Tabulaciones

In [153]:
correlation_matrix = dfDIT[['apego9a12', 'apego9a12_cat', 'comEfec9a12', 'comEfec9a12_cat']].corr()
correlation_matrix

,apego9a12,apego9a12_cat,comEfec9a12,comEfec9a12_cat
apego9a12,1.000000,0.811155,-0.026716,-0.007959
apego9a12_cat,0.811155,1.000000,0.002029,0.047939
comEfec9a12,-0.026716,0.002029,1.000000,0.881844
comEfec9a12_cat,-0.007959,0.047939,0.881844,1.000000


In [149]:
dfDIT['comEfec9a12_cat'].value_counts()

comEfec9a12_cat
3.0    4014
2.0     895
1.0     232
0.0     108
Name: count, dtype: int64

In [126]:
dfDIT['qi478e7'].value_counts()

qi478e7
No                     3947
Si                     1255
No responde/No sabe       2
Name: count, dtype: int64

In [128]:
dfDIT['precursorApego9a12'].value_counts()

precursorApego9a12
0.0    2918
1.0    2331
Name: count, dtype: int64

In [ ]:
# Factor de expansion
merged_final["wt"] = merged_final["v005"] / 1000000

# Tabulations
tab1 = pd.crosstab(merged_final["pesonac"], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab2 = pd.crosstab([merged_final["v463a"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab3 = pd.crosstab([merged_final["v190"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab4 = pd.crosstab([merged_final["etnia"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab5 = pd.crosstab([merged_final["lmaterna"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab6 = pd.crosstab([merged_final["v025"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab7 = pd.crosstab([merged_final["sregion"], merged_final["pesonac"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")

# Tamaño de la niña o niño al nacer
merged_final["ch_size_birth"] = pd.cut(merged_final["m18"], bins=[-1, 2, 4, 5, 8, 9], labels=["Promedio o mayor", "Menor que promedio", "Muy pequeño", "No sabe/sin información"], right=False)
tab8 = pd.crosstab(merged_final["ch_size_birth"], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab9 = pd.crosstab([merged_final["v463a"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab10 = pd.crosstab([merged_final["v190"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab11 = pd.crosstab([merged_final["etnia"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab12 = pd.crosstab([merged_final["lmaterna"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab13 = pd.crosstab([merged_final["v025"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
tab14 = pd.crosstab([merged_final["sregion"], merged_final["ch_size_birth"]], merged_final["e_0a59"], values=merged_final["wt"], aggfunc="sum")
